# Reading silver table

In [0]:
from delta.tables import DeltaTable

In [0]:
df_silver = spark.sql("SELECT * FROM PARQUET.`abfss://silver@datalakeproject1goks.dfs.core.windows.net/car_sales`")

In [0]:
df_silver.display()

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Day,Month,Year,BranchName,DealerName,mode_category,Rev_Per_Unit
BR0001,DLR0001,BMW-M1,13363978,2,DT00001,1,1,2017,AC Cars Motors,AC Cars Motors,BMW,6681989.0
BR0003,DLR0228,Hon-M218,17376468,3,DT00001,10,5,2017,AC Cars Motors,Deccan Motors,Hon,5792156.0
BR0004,DLR0208,Tat-M188,9664767,3,DT00002,12,1,2017,AC Cars Motors,Wiesmann Motors,Tat,3221589.0
BR0005,DLR0188,Hyu-M158,5525304,3,DT00002,16,9,2017,AC Cars Motors,Subaru Motors,Hyu,1841768.0
BR0006,DLR0168,Ren-M128,12971088,3,DT00003,20,5,2017,AC Cars Motors,Saab Motors,Ren,4323696.0
BR0008,DLR0128,Hon-M68,7321228,1,DT00004,28,4,2017,AC Cars Motors,Messerschmitt Motors,Hon,7321228.0
BR0009,DLR0108,Cad-M38,11379294,2,DT00004,31,12,2017,AC Cars Motors,Lexus Motors,Cad,5689647.0
BR0010,DLR0088,Mer-M8,11611234,2,DT00005,4,9,2017,AC Cars Motors,"IFA (including Trabant, Wartburg, Barkas) Motors",Mer,5805617.0
BR0011,DLR0002,BMW-M2,19979446,2,DT00005,2,1,2017,Acura Motors,Acura Motors,BMW,9989723.0
BR0011,DLR0069,Vol-M256,14181510,3,DT00006,9,5,2017,Acura Motors,Geo Motors,Vol,4727170.0


Reading all DIMS

In [0]:
df_dealer = spark.sql("SELECT* FROM cars_catalog.gold.dim_dealer")
df_branch = spark.sql("SELECT* FROM cars_catalog.gold.dim_branch")
df_model = spark.sql("SELECT* FROM cars_catalog.gold.dim_model")
df_date = spark.sql("SELECT* FROM cars_catalog.gold.dim_date")

Bringing keys to fact table

In [0]:
df_fact = df_silver.join(df_branch,df_silver['Branch_ID']==df_branch['Branch_ID'], how ='left')\
    .join(df_dealer,df_silver['Dealer_ID']==df_dealer['Dealer_ID'],how ='left')\
     .join(df_date,df_silver['Date_ID']==df_date['Date_ID'],how ='left')\
     .join(df_model,df_silver['Model_ID']==df_model['Model_ID'],how ='left')\
     .select(df_silver.Revenue, df_silver.Rev_Per_Unit,df_silver.Units_Sold,df_branch.dim_branch_key,df_dealer.dim_dealer_key, df_model.dim_model_key, df_date.dim_date_key)

In [0]:
df_fact.display()

Revenue,Rev_Per_Unit,Units_Sold,dim_branch_key,dim_dealer_key,dim_model_key,dim_date_key
13363978,6681989.0,2,465,118,22,666
17376468,5792156.0,3,1220,179,110,666
9664767,3221589.0,3,588,131,97,693
5525304,1841768.0,3,5,208,25,693
12971088,4323696.0,3,407,223,214,192
7321228,7321228.0,1,1813,171,247,597
11379294,5689647.0,2,267,214,125,597
11611234,5805617.0,2,847,63,225,745
19979446,9989723.0,2,1178,267,5,745
14181510,4727170.0,3,1178,129,23,800


In [0]:
#Incremental Run
 # Deduplicate source data based on merge keys
df_fact = df_fact.dropDuplicates(['dim_branch_key', 'dim_model_key', 'dim_date_key', 'dim_dealer_key'])
if spark.catalog.tableExists('cars_catalog.gold.fact_sales'):
    delta_table= DeltaTable.forPath(spark, 'abfss://gold@datalakeproject1goks.dfs.core.windows.net/fact_sales')
    delta_table.alias('trg').merge(df_fact.alias('src'), "src.dim_branch_key == trg.dim_branch_key and src.dim_model_key==trg.dim_model_key and src.dim_date_key==trg.dim_date_key and src.dim_dealer_key==trg.dim_dealer_key")\
                            .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll()\
                            .execute()

#First Run
else:
    df_fact.write.format('delta')\
                  .mode('overwrite')\
                  .option("path",'abfss://gold@datalakeproject1goks.dfs.core.windows.net/fact_sales')\
                  .saveAsTable("cars_catalog.gold.fact_sales")

In [0]:
%sql
SELECT * FROM cars_catalog.gold.fact_sales

Revenue,Rev_Per_Unit,Units_Sold,dim_branch_key,dim_dealer_key,dim_model_key,dim_date_key
13363978,6681989.0,2,465,118,22,666
17376468,5792156.0,3,1220,179,110,666
9664767,3221589.0,3,588,131,97,693
5525304,1841768.0,3,5,208,25,693
12971088,4323696.0,3,407,223,214,192
7321228,7321228.0,1,1813,171,247,597
11379294,5689647.0,2,267,214,125,597
11611234,5805617.0,2,847,63,225,745
19979446,9989723.0,2,1178,267,5,745
14181510,4727170.0,3,1178,129,23,800
